## Packages and Libraries

In [ ]:
!pip install searchtweets
!pip install GetOldTweets3
!pip install snscrape
!pip install PyDrive
!pip install snorkel
!pip install networkx
!pip install DateTime

In [ ]:
# utilities, dataFrame, and API packages
import json, os, requests, oauthlib, time, urllib, urllib.request, csv, zipfile, io, shutil, re, scipy, jinja2, urllib3
import pandas as pd # package that allows us to make and manipulate dataFrames
from pandas.io.json import json_normalize # package allows us to "flatten" json files, eg, extract nested data in json files
from pprint import pprint # print output in a readable fashion
import lxml
import http.client
from datetime import datetime
from http.client import IncompleteRead
from urllib.parse import urlparse, parse_qs, urlsplit, unquote # allows us to parse and work with URLs
from DateTime import DateTime
import re #regular expression package 
import csv # read/write csv package 
import string
import time

# statistical computing (including ML) and working with numbers
import numpy as np 
import scipy.stats as stats # stastistics package 
import glob # NLP package

# twitter API and data packages
from searchtweets import ResultStream, gen_rule_payload, load_credentials 
import GetOldTweets3 as got
import snscrape
from snscrape import modules
from snscrape.modules import twitter
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import requests

# data visualization packages
import seaborn as sns 
import matplotlib.pyplot as plt 

# Google Drive packages
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

## Searching For Tweets w/ Keywords and Phrases

In [ ]:
# Use snscrape to pull tweet urls containing key words and phases
'''
!snscrape --since [start date] twitter-search "[keyword/phrase] until:[end date]" > [save file path]
''''

## Pulling Tweets and Reference Tweets




## Function Library

* replaceBrackets(tweetIDs)
  * tweetIDs is formated as a 2D list, where each element is a list with one string eleemnt holding an ID 
    * ie. [['123'], ['456'], ...]
  * Returns a list of tweet IDs as strings 
    * ie. ['123', '456', ...]
* make100(tweetList, n)
  * tweetList is formatted as a list of tweet IDs as a string
    * ie. ['123', '456', ...]
  * Returns the nth list of 100 tweet IDs
* convertFormat(tweets)
  * tweets is formatted as a list of tweet IDs as a string
    * ie. ['123', '456', ...]
  * Returns the provided tweet IDs as a concatenated string separated by commas
    * ie. "123,456,..."
* pull_data(tweet_ids, n)
  * tweet_ids is formatted as a list of tweet IDs as a string
    * ie. ['123', '456', ...]
  * n = level in tree (ie. 0 is first scrape, 1 is second, 2 is third, etc)
  * Requests tweet data from Twitter's API for given list of tweet IDs in groups of 100, organizes it, and stores it as a CSV file
  * Depends on: make100(), convertFormat()
* merge_files(wdir, filename_format, save_path)
  * Merges all csv files following specified file_format in wdir into one and saves to save_path
* parse_dict(s)
  * s is a string containing a list containing a dictionary
    * ie. "[{'key': "data"}]"
  * Returns the dictionary contained within the string
* label_toplevel(file_path)
  * Adds a top_level column to the dataframe and tags the toplevel tweets
* convert_time_format(file_path, save_path)
  * Takes data in created_at column and formats it for easy use
  * Note: Converts a given csv file to a pandas dataframe and saves as a csv file
* extract_above_level(file_path)
  * Extracts tweet id of referenced tweet from dictionary returned by parse_dict()
  * Stores and returns ids in a list formatted for pull_data()
  * Depends on parse_dict()
* not_all_toplevel(file_path)
  * Returns true if given tweets includes non-toplevel tweets, and false otherwise




In [ ]:
# Written by Proteeti Sinha

# tweetIDs is formated as a 2D list, where each element is a list with one string eleemnt holding an ID
# ie. [['123'], ['456'], ...]

# Returns a list of tweet IDs as strings
# ie. ['123', '456', ...]

def replaceBrackets(tweetIDs): 
    result = [] 
    for tweetID in tweetIDs:
        tweetID = str(tweetID)
        tweetID = tweetID.replace("['", "")
        tweetID = tweetID.replace("']", "")
        result.append(tweetID)
    return result

In [ ]:
# Written by Proteeti Sinha

# tweetList is formatted as a list of tweet IDs as a string
# ie. ['123', '456', ...]

# Returns the nth list of 100 tweet IDs

def make100(tweetList, n):
    result = []
    current = n * 100
    listLen = len(tweetList)
    if (listLen - current) < 100:
        for i in range(current, listLen):
            result.append(tweetList[i])
        return result
    else:
        for i in range(current, current + 100):
            result.append(tweetList[i])
        return result 
    print("ERROR")

In [ ]:
# Written by Proteeti Sinha

# tweets is formatted as a list of tweet IDs as a string
# ie. ['123', '456', ...]

# Returns the provided tweet IDs as a concatenated string separated by commas
# ie. "123,456,..."

def convertFormat(tweets):
    result = ""
    for tweet in range(len(tweets)-1):
        # print(tweets[tweet])
        numConvert = str(int(tweets[tweet]))
        result = result + numConvert + ","
    result = result + str(int(tweets[-1]))
    return result

In [ ]:
# tweet_ids is formatted as a list of tweet IDs as a string
# ie. ['123', '456', ...]

# n = level in tree (ie. 0 is first scrape, 1 is second, 2 is third, etc)

# Requests tweet data from Twitter's API for given list of tweet IDs in groups of 100, organizes it, and stores it as a CSV file
# Depends on: make100(), convertFormat()

def pull_data(tweet_ids, n):

  # TODO: Change authorization credentials
  headers = {
      'Authorization': ''
  }

  # Calculates the number of groups of a hundred ids in tweet_ids
  x = len(tweet_ids) / 100
  num_of_hundreds = int(x)
  if x > num_of_hundreds: num_of_hundreds += 1

  # Debugging printlines
  print("Length of tweet_ids: ", len(tweet_ids))
  print("Number of groups of a hundred:", num_of_hundreds)

  # For loop automates process of pulling data for groups of 100
  for i in range(num_of_hundreds):
    print("Hundred #: ", i)
    hundo = make100(tweet_ids, i)

    params = (
      ('ids', convertFormat(hundo)),
      ('expansions', 'author_id,attachments.poll_ids,attachments.media_keys,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id'),
      ('place.fields','contained_within,country,country_code,full_name,geo,id,name,place_type'),
      ('poll.fields','duration_minutes,end_datetime,id,options,voting_status'),
      ('tweet.fields', 'public_metrics,attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,referenced_tweets,reply_settings,source,text,withheld'),
      ('media.fields', 'duration_ms,height,media_key,preview_image_url,type,url,width,public_metrics'),
      ('user.fields','created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'),
    )

    # API Request
    response = requests.get('https://api.twitter.com/2/tweets', headers=headers, params=params)
    time.sleep(5)

    c = json.loads(response.text)
    cc=c['data']
    c_norm=pd.json_normalize(cc)

    # Creates an empty dataframe that is then appended on to
    # User has control over column inclusion and order
    extractedDF = pd.DataFrame(c_norm)

    tweetDF = pd.DataFrame(columns = ['id', 'created_at', 'text', 
                     'conversation_id', 'author_id', 'referenced_tweets', 
                     'in_reply_to_user_id', 'lang', 'possibly_sensitive', 
                     'reply_settings', 'source', 'context_annotations', 
                     'public_metrics.like_count', 'public_metrics.retweet_count', 'public_metrics.reply_count',
                     'public_metrics.quote_count', 'attachments.media_keys', 'attachments.poll_ids',
                     'entities.hashtags', 'entities.urls', 'entities.mentions',
                     'entities.annotations', 'geo.place_id'])
    
    tweetDF = tweetDF.append(extractedDF)

    tweetDF.to_csv(f'/content/drive/MyDrive/Research/CHIMPS/(Data) Portrait AI/Iteration 3/PAI Tweets/100 Groups/PAI_{n}.{i}.csv')

In [ ]:
# Merges all csv files following specified file_format in wdir into one and saves to save_path

def merge_files(wdir, file_format, save_path):
  # https://blog.softhints.com/how-to-merge-multiple-csv-files-with-python/#step1importmodulesandsettheworkingdirectory
  all_files = glob.glob(os.path.join(wdir, file_format))
  df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
  df_merged = pd.concat(df_from_each_file, ignore_index=True)
  df_merged.to_csv(save_path)

In [ ]:
# s is a string containing a list containing a dictionary
# ie. "[{'key': "data"}]"

# Returns the dictionary contained within the string

def parse_dict(s): 
    # https://www.w3schools.com/python/ref_string_replace.asp
    one = s.replace("[", "")
    two = one.replace("]", "")

    # https://www.geeksforgeeks.org/python-program-to-create-a-dictionary-from-a-string/
    d = eval(two)
    
    return(d)

In [ ]:
# Adds a top_level column to the dataframe and tags the toplevel tweets

def label_toplevel(file_path):
  df = pd.read_csv(file_path)

  replies_df = pd.get_dummies(df, dummy_na=True, columns=pd.Series('in_reply_to_user_id'))
  top_level = replies_df['in_reply_to_user_id_nan']
  df['top_level'] = top_level

  df.to_csv(file_path)

In [ ]:
# Takes data in created_at column and formats it for easy use
# Note: Converts a given csv file to a pandas dataframe and saves as a csv file

def convert_time_format(file_path, save_path):
  df = pd.read_csv(file_path)
  num_rows = len(df)

  created_at_col_idx = df.columns.get_loc('created_at')

  for i in range(num_rows):
    created_datetime = df.iloc[i, created_at_col_idx]
    # https://pypi.org/project/DateTime/
    x = DateTime(created_datetime)
    df.iloc[i, created_at_col_idx] = x.ISO()

  df.to_csv(save_path)

In [ ]:
# Extracts tweet id of referenced tweet from dictionary returned by parse_dict()
# Stores and returns ids in a list formatted for pull_data()

def extract_above_level(file_path):
  df = pd.read_csv(file_path)
  num_rows = len(df)

  ref_tweets_col_idx = df.columns.get_loc('referenced_tweets')

  to_extract_ids = []

  for i in range(num_rows):
    # print(i)
    # https://towardsdatascience.com/how-to-use-loc-and-iloc-for-selecting-data-in-pandas-bd09cb4c3d79
    ref_tweet_info = df.iloc[i, ref_tweets_col_idx]
    if type(ref_tweet_info) == str:
      info_dict = parse_dict(ref_tweet_info)
      if type(info_dict) != dict: continue
      id = info_dict['id']
      to_extract_ids.append(id)
  
  return to_extract_ids

In [ ]:
# Returns true if given tweets includes non-toplevel tweets, and false otherwise

def not_all_toplevel(file_path):
  df = pd.read_csv(file_path)
  num_rows = len(df)

  toplevel_col_idx = df.columns.get_loc('top_level')

  for i in range(num_rows):
    is_toplevel = df.iloc[i, toplevel_col_idx]
    if is_toplevel == 0: return True
  return False

Filename Library

"PAI_a.b" --> files storing raw data for groups of 100 (a = level #, b = file #)

"PAI_merged_a" --> files storing merged

"PAI_all_a" --> files storing engineered data for level a

## Main Function

In [ ]:
def main():
  # TODO: Add scraped tweet urls path 
  tweet_urls = ""

  # TODO: Add path of folder where all scraped data will be saved
  working_directory = ""

  # TODO: Add a file name format
  # i.e. "[data set name]_0.*.csv"
  # file names will update to be "[data set name]_[level].[nth 100].csv"
  file_format = ""

  # TODO: Add a merged file path and name (where the groups of 100 from one level are merged together)
  # i.e. "[data set name]_merged_0.csv"
  # file names will update to be "[data set name]_merged_[level].csv"
  merged_file_path = "" 

  # TODO: Add a save file path and name (where a copy of the merged files are saved)
  # i.e. "[data set name]_all_0.csv"
  # file names will update to be "[data set name]_all_[level].csv"
  save_path = ""

  # CSV --> pandas dataframe
  tweets = pd.read_csv(tweet_urls, header=None)

  # Written Proteeti Sinha
  # Parsing out tweet ids from urls
  urlz=tweets[0]
  tweetIDs = []
  for url in urlz:
    tweetRegex = re.findall(r'\d\d\d\d\d\d\d\d\d\d\d+', url)
    tweetIDs.append(tweetRegex)

  # Configuring 2D list of IDs as strings to a list of IDs as strings
  tweetListOverall = replaceBrackets(tweetIDs)

  print("Level: 0")

  # Pulling data from urls give above
  pull_data(tweetListOverall, 0)

  # Merging all the resulting files and saving it into one file
  merge_files(working_directory, file_format, merged_file_path)

  # Extracts tweet ids from level above
  above_level_tweet_ids = extract_above_level(merged_file_path) # updates as loop runs
  # Labels the toplevel tweets
  label_toplevel(merged_file_path)
  # Converts the time formatting and saves it into another file
  convert_time_format(merged_file_path, save_path)

  print("Level: 0 finish")
  print()

  # Runs after first base data set of urls are done pulling
  # Collects and pulls data for higher level tweets, until all toplevel tweets are found and included
  count = 1 # level 1

  while not_all_toplevel(save_path) and len(above_level_tweet_ids) > 0:
    print("Level: ", count)
    pull_data(above_level_tweet_ids, count)

    # TODO: Add a file name format
    # i.e. "[data set name]_{count}.*.csv"
    # file names will update to be "[data set name]_[level].[nth 100].csv"
    file_format = f""

    # TODO: Add a merged file path and name (where the nth data from one level are merged together)
    # i.e. "[data set name]_merged_{count}.csv"
    # file names will update to be "[data set name]_merged_[level].csv"
    merged_file_path = f""

    # TODO: Add a save file path and name (where a copy of the merged files are saved)
    # i.e. "[data set name]_all_{count}.csv"
    # file names will update to be "[data set name]_all_[level].csv"
    save_path = f""

    merge_files(working_directory, file_format, merged_file_path)

    above_level_tweet_ids = extract_above_level(merged_file_path)
    label_toplevel(merged_file_path)
    convert_time_format(merged_file_path, save_path)

    print("Level: ", count, "finish")
    print()
    count += 1


In [ ]:
# Run after updating paths in main() and pull_data()

main()

## Merge all the generated files into one

In [ ]:
# Merge all the final data files and remove unnecessary columns

# TODO: Add path to folder saving files with "[data set name]_all_[level].csv"
wdir = ""
# TODO: Add filename format "[data set name]_all_*.csv"
filename_format = ""
# TODO: Add save file name and 
save_path = ""

merge_files(wdir, filename_format, save_path)